### Spotify Initial 2021 Data Collection, Observation, and Cleaning ###

This code takes advantage of Spotipy, a package which allows one to use Spotify's API, to gather track-level data maintained by Spotify. It produces a dataset with information on each of the top 50 most popular songs on Spotify in America in 2021, as measured by Spotify and released to the public via an [ordered playlist](https://open.spotify.com/user/spotify/playlist/37i9dQZF1DXaqCgtv7ZR3L?si=K4BLJsyXSy-j0cSED9FjgQ) at the end of the year.

Afterwards, the dataset is observed and cleaned.

Source: Spotify Web API, [Spotify Top Tracks of 2021 USA](https://open.spotify.com/playlist/37i9dQZF1DXbJMiQ53rTyJ?si=b42dbe5c50d545a3)

Downloaded: 12/13/2021

Srinidhi Ramakrishna

In [1]:
# Importing packages
import spotipy
import time
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Locating my unique client and secret ID as a developer
cid = '303cbc1ce8224ed0987dae7b34810613'
secret = '91b877b5e37d44529d31a35da6e9461e'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
# Collecting track IDs based on the playlist URL
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids = getTrackIDs('spotify', '37i9dQZF1DXbJMiQ53rTyJ')

In [4]:
# Collecting track features for each song
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  duration_ms = meta['duration_ms']
  popularity = meta['popularity']
  explicit = meta['explicit']
    
  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']


  track = [name, album, artist, release_date, duration_ms, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence, explicit]
  return track

In [5]:
# Looping over track ids to append track-level metrics in a new row 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

In [6]:
# Creating dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'explicit'])
df.to_csv("../../data/Raw/spotify2021raw.csv", sep = ',')

#### Data Observation

In [7]:
spotifyraw2021 = pd.read_csv("../../data/Raw/spotify2021raw.csv")

Let's take a basic look at the dimensions of this dataset, as well as the meanings of the rows and columns. 

In [8]:
spotifyraw2021.shape

(50, 19)

In [9]:
spotifyraw2021.head()

,Unnamed: 0,name,album,artist,release_date,duration_ms,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,explicit
0,0,drivers license,SOUR,Olivia Rodrigo,2021-05-21,242013,93,0.561,0.76800,0.561,0.431,0.000014,0.1060,-8.810,0.0578,143.875,4,0.137,True
1,1,good 4 u,SOUR,Olivia Rodrigo,2021-05-21,178146,96,0.563,0.33500,0.563,0.664,0.000000,0.0849,-5.044,0.1540,166.928,4,0.688,True
2,2,Kiss Me More (feat. SZA),Planet Her,Doja Cat,2021-06-25,208666,88,0.764,0.25900,0.764,0.705,0.000089,0.1200,-3.463,0.0284,110.970,4,0.781,True
3,3,Heat Waves,Dreamland (+ Bonus Levels),Glass Animals,2020-08-06,238805,94,0.761,0.44000,0.761,0.525,0.000007,0.0921,-6.900,0.0944,80.870,4,0.531,False
4,4,Levitating (feat. DaBaby),Future Nostalgia,Dua Lipa,2020-03-27,203064,89,0.702,0.00883,0.702,0.825,0.000000,0.0674,-3.787,0.0601,102.977,4,0.915,False


This dataset has 50 rows and 19 columns, with each row representing a song. Rows are ordered in accordance with popularity in the US in 2017 as determined by Spotify. The columns represent a variety of track-level metrics. Rows and columns in this dataset are the same as those in the 2017 and 2020 Spotify datasets, and full descriptions of the meaning of these rows and columns can be found in the notebooks "Spotify Initial 2017 Data Collection, Observation, and Cleaning.ipynb" and "Spotify Initial 2020 Data Collection, Observation, and Cleaning.ipynb."

#### Data Cleaning

Here, I perform the same data cleaning steps that were performed on the 2017 and 2020 Spotify datasets. Explanations for these steps can be found in the same aforementioned notebooks. 

In [10]:
spotifytop2021 = spotifyraw2021.drop(['popularity', 'time_signature', 'danceability.1'], axis = 1)


In [11]:
spotifytop2021 = spotifytop2021.rename(columns = {"Unnamed: 0": "rank"})
spotifytop2021['rank'] = spotifytop2021['rank'] + 1

In [12]:
spotifytop2021['duration_sec'] = spotifytop2021['duration_ms']/1000
spotifytop2021 = spotifytop2021.drop(['duration_ms'], axis = 1)

In [13]:
spotifytop2021.sample(10)

,rank,name,album,artist,release_date,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,explicit,duration_sec
1,2,good 4 u,SOUR,Olivia Rodrigo,2021-05-21,0.563,0.33500,0.664,0.000000,0.0849,-5.044,0.1540,166.928,0.6880,True,178.146
5,6,MONTERO (Call Me By Your Name),MONTERO,Lil Nas X,2021-09-17,0.593,0.29300,0.503,0.000000,0.4050,-6.725,0.2200,178.781,0.7100,True,137.704
40,41,Circles,Hollywood's Bleeding,Post Malone,2019-09-06,0.695,0.19200,0.762,0.002440,0.0863,-3.497,0.0395,120.042,0.5530,False,215.280
41,42,What You Know Bout Love,Shoot For The Stars Aim For The Moon,Pop Smoke,2020-07-03,0.709,0.65000,0.548,0.000002,0.1330,-8.493,0.3530,83.995,0.5430,True,160.000
25,26,Sweater Weather,I Love You.,The Neighbourhood,2013-04-19,0.612,0.04950,0.807,0.017700,0.1010,-2.810,0.0336,124.053,0.3980,False,240.400
7,8,RAPSTAR,Hall of Fame,Polo G,2021-06-11,0.789,0.41000,0.536,0.000000,0.1290,-6.862,0.2420,81.039,0.4370,True,165.925
17,18,traitor,SOUR,Olivia Rodrigo,2021-05-21,0.380,0.69100,0.339,0.000000,0.1200,-7.885,0.0338,100.607,0.0849,False,229.226
43,44,Laugh Now Cry Later (feat. Lil Durk),Laugh Now Cry Later (feat. Lil Durk),Drake,2020-08-14,0.761,0.24400,0.518,0.000035,0.1070,-8.871,0.1340,133.976,0.5220,True,261.492
26,27,Best Friend (feat. Doja Cat),Best Friend (feat. Doja Cat) [Remix EP] [Exten...,Saweetie,2021-04-23,0.840,0.00302,0.766,0.000004,0.0684,-4.120,0.1360,94.018,0.4020,True,155.883
45,46,positions,Positions,Ariana Grande,2020-10-30,0.737,0.46800,0.802,0.000000,0.0931,-4.771,0.0878,144.015,0.6820,True,172.324


With a few small changes, the dataset is more readable!

In [14]:
spotifytop2021.to_csv("../../data/Clean/spotifytop2021cleaned.csv", sep = ',')